In [13]:
import torch
import numpy as np
from scipy import signal

In [2]:
image_eeg_path = r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\eeg_signals_raw_with_mean_std.pth"

image_eeg = torch.load(image_eeg_path)
print(image_eeg.keys())

dict_keys(['dataset', 'labels', 'images', 'means', 'stddevs'])


In [3]:
for i in image_eeg['dataset']:
    assert i['eeg'].float().t()[20:460, :].shape[0] == 440

In [47]:
# reshape a torch tensor of shape (128, 440) to a shape (128, )

In [4]:
dataset_dict = {
    'rawData': image_eeg['dataset'],
    'mean': image_eeg['means'],
    'std': image_eeg['stddevs'],
    'labels' : image_eeg['labels'],
    'images': image_eeg['images'],
}

In [5]:
processed_data = []

for i in dataset_dict['rawData']:
    processed_data.append({
        'eeg': i['eeg'].float()[:, 20:460],
        'label': i['label'],
        'image': i['image']
    })

In [6]:
def preprocess_eeg(eeg_data, original_sfreq, target_sfreq, lowcut=0.5, highcut=75, notch_freq=50, notch_width=3):
    """
    Preprocesses EEG data by applying bandpass and notch filters, and then resamples it.

    Args:
        eeg_data (numpy.ndarray): EEG data with shape (num_channels, num_samples).
        original_sfreq (float): Original sampling frequency of the EEG data.
        target_sfreq (float): Target sampling frequency after resampling.
        lowcut (float): Lower cutoff frequency for the bandpass filter (Hz).
        highcut (float): Upper cutoff frequency for the bandpass filter (Hz).
        notch_freq (float): Frequency to notch filter (Hz).
        notch_width(float) : Width of the notch filter in Hz.

    Returns:
        numpy.ndarray: Preprocessed and resampled EEG data with shape (num_channels, resampled_samples).
    """

    num_channels, num_samples = eeg_data.shape

    # 1. Bandpass Filter
    nyquist = 0.5 * original_sfreq
    low = lowcut / nyquist
    high = highcut / nyquist
    b_band, a_band = signal.butter(5, [low, high], btype='band')  # 5th order butterworth filter

    filtered_data = np.zeros_like(eeg_data, dtype=np.float64)
    for channel in range(num_channels):
        filtered_data[channel, :] = signal.lfilter(b_band, a_band, eeg_data[channel, :])

    # 2. Notch Filter
    notch_q = 20
    b_notch, a_notch = signal.iirnotch(notch_freq, notch_q, original_sfreq)

    notched_data = np.zeros_like(filtered_data, dtype=np.float64)
    for channel in range(num_channels):
        notched_data[channel, :] = signal.lfilter(b_notch, a_notch, filtered_data[channel, :])

    # 3. Resampling
    resampled_samples = int(num_samples * target_sfreq / original_sfreq)
    resampled_data = np.zeros((num_channels, resampled_samples), dtype=np.float64)

    for channel in range(num_channels):
        resampled_data[channel, :] = signal.resample(notched_data[channel, :], resampled_samples)

    return resampled_data

In [7]:
# Apply preprocessing to the EEG data in processed_data
for i in range(len(processed_data)):
    eeg_data = processed_data[i]['eeg'].numpy()
    processed_data[i]['eeg'] = preprocess_eeg(eeg_data, original_sfreq=1000, target_sfreq=200)
    # print("Shape of EEG data after preprocessing:", processed_data[i]['eeg'].shape)

# dataset_dict['processed'] = processed_data

# np.save(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", dataset_dict, allow_pickle=True)

In [8]:
# check shape of eeg data is (128, 88)
for i in processed_data:
    assert i['eeg'].shape[0] == 128, f"Shape of EEG data is not correct: {i['eeg'].shape}"
    assert i['eeg'].shape[1] == 88, f"Shape of EEG data is not correct: {i['eeg'].shape}"

In [9]:
# reshape each np array of shape (128, 88) to a torch tensor of shape (128, 1, 88) for processed_data
for i in range(len(processed_data)):
    processed_data[i]['eeg'] = torch.from_numpy(processed_data[i]['eeg']).unsqueeze(1)


In [10]:
dataset_dict['processed'] = processed_data

In [11]:
np.save(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", dataset_dict, allow_pickle=True)

In [12]:
# create a mask for the data, put 1 wherever masking is required
for i in range(len(processed_data)):
    if processed_data[i]['eeg'].shape[2] < 200:
        pad_size = 200 - processed_data[i]['eeg'].shape[2]
        processed_data[i]['mask'] = torch.ones(1, 200).float()
        processed_data[i]['mask'][:, :processed_data[i]['eeg'].shape[2]] = 1e-9
        processed_data[i]['eeg'] = torch.nn.functional.pad(processed_data[i]['eeg'], (0, pad_size), 'constant', 0)
    # elif processed_data[i]['eeg'].shape[2] > 200:
    #     processed_data[i]['mask'] = torch.ones(1, 200).float()
    #     processed_data[i]['mask'][:, :processed_data[i]['eeg'].shape[2]] = 0
    # else:
    #     processed_data[i]['mask'] = torch.ones(1, 200).float()

# pad the eeg data to 200 samples
# for i in range(len(processed_data)):
#     if processed_data[i]['eeg'].shape[2] < 200:
#         pad_size = 200 - processed_data[i]['eeg'].shape[2]
#         processed_data[i]['eeg'] = torch.nn.functional.pad(processed_data[i]['eeg'], (0, pad_size), 'constant', 0)
    # elif processed_data[i]['eeg'].shape[2] > 200:
    #     processed_data[i]['eeg'] = processed_data[i]['eeg'][:, :, :200]

# save the processed data to a numpy file
dataset_dict['processed'] = processed_data
np.save(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", dataset_dict, allow_pickle=True)

In [56]:
# load the processed data from the numpy file and print the shape of the data and mask
dataset_dict = np.load(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", allow_pickle=True).item()
print("Shape of EEG data after preprocessing:", dataset_dict['processed'][0]['eeg'].shape)
print("Shape of EEG mask after preprocessing:", dataset_dict['processed'][0]['mask'].shape)

Shape of EEG data after preprocessing: torch.Size([128, 1, 200])
Shape of EEG mask after preprocessing: torch.Size([1, 200])


In [57]:
dataset_dict.keys()

dict_keys(['rawData', 'mean', 'std', 'labels', 'images', 'processed'])

In [58]:
dataset_dict['processed'][0]['eeg']

tensor([[[ 6.2282e+00, -2.7308e+00, -1.1331e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 3.9343e+00, -3.6040e+00, -6.5825e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.9413e+00,  3.2425e+00,  6.1431e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        ...,

        [[ 1.4755e+02, -2.4078e+02, -8.7423e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 1.3477e+02, -2.3237e+02, -8.7572e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 3.6133e+02, -6.7821e+02, -2.4972e+03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]], dtype=torch.float64)

In [59]:
half_processed = []

for i in dataset_dict['rawData']:
    half_processed.append({
        'eeg': i['eeg'].float()[:, 20:460].unsqueeze(1),
        'label': i['label'],
        'image': i['image']
    })

dataset_dict['half_processed'] = half_processed

np.save(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", dataset_dict, allow_pickle=True)

In [2]:
# load various processed and half processed data and print the shape of the data and mask
dataset_dict = np.load(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", allow_pickle=True).item()
print("Shape of EEG data after preprocessing:", dataset_dict['processed'][0]['eeg'].shape)
print("Shape of EEG mask after preprocessing:", dataset_dict['processed'][0]['mask'].shape)
print("Shape of non-resampled EEG data:", dataset_dict['half_processed'][0]['eeg'].shape)

Shape of EEG data after preprocessing: torch.Size([128, 1, 200])
Shape of EEG mask after preprocessing: torch.Size([1, 200])
Shape of non-resampled EEG data: torch.Size([128, 1, 440])


In [3]:
# remove the key 'half_processed' from the dataset_dict
del dataset_dict['half_processed']

# save the dataset_dict again
np.save(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", dataset_dict, allow_pickle=True)

# Subject analysis

In [14]:
data = np.load(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", allow_pickle=True).item()
data.keys()

dict_keys(['rawData', 'mean', 'std', 'labels', 'images', 'processed'])

In [16]:
data['rawData'][0]['subject']

4

# Montage analysis

In [1]:
import mne
from mne.viz import set_3d_title, set_3d_view
import numpy as np

In [4]:
%pip install ipywidgets pyvistaqt ipyevents trame

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
montage_path = r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\actiCAP_snap_CACS_CAS_GACS-v2\actiCap_snap_CACS_CAS_GACS\actiCap_slim_for actiChamp_Plus\CACS-128\CACS-128_NO_REF.bvef"

In [6]:
montage = mne.channels.read_custom_montage(montage_path)
len(montage.ch_names[1:])

128

# Synset Analysis


In [1]:
import numpy as np

In [2]:
data = np.load(r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\processed_eeg_signals.npy", allow_pickle=True).item()
data.keys()

dict_keys(['rawData', 'mean', 'std', 'labels', 'images', 'processed'])

In [3]:
unique_synsets = list(set([i.strip().split('_')[0].strip() for i in data['images']]))
unique_synsets

['n03773504',
 'n02389026',
 'n02504458',
 'n03445777',
 'n03376595',
 'n03584829',
 'n03590841',
 'n02510455',
 'n04044716',
 'n03100240',
 'n03297495',
 'n13054560',
 'n02992529',
 'n04086273',
 'n02690373',
 'n02124075',
 'n02607072',
 'n03180011',
 'n03775071',
 'n02951358',
 'n03272010',
 'n03792782',
 'n04069434',
 'n03197337',
 'n02106662',
 'n03452741',
 'n03792972',
 'n07873807',
 'n03709823',
 'n02281787',
 'n02492035',
 'n03888257',
 'n11939491',
 'n02906734',
 'n03877472',
 'n03272562',
 'n07753592',
 'n04120489',
 'n03982430',
 'n03063599']

In [8]:
%pip install wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9711 sha256=b3a63fbfcd7820fa7edc7c5b150ead122649077fb8de58489b75bcf875ab2873
  Stored in directory: c:\users\vijayh\appdata\local\pip\cache\wheels\8a\b8\04\0c88fb22489b0c049bee4e977c5689c7fe597d6c4b0e7d0b6a
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [4]:
links = []

for i in unique_synsets:
    links.append(f"https://image-net.org/data/winter21_whole/{i}.tar")

In [6]:
out_path = r"D:\Vijay\NYU\Spring_25\BDMLS\Project\dataset\ImageNetEEG\images"

import os
import wget

for i in unique_synsets:
    link = f"https://image-net.org/data/winter21_whole/{i}.tar"
    try:
        filename = wget.download(i, out=os.path.join(out_path, i + ".tar"))
    except Exception as e:
        print(f"Failed to download {i}: {e}")

Failed to download n03773504: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n03773504.tardln4kod_.tmp'
Failed to download n02389026: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n02389026.tarz706l1tn.tmp'
Failed to download n02504458: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n02504458.tarhxew3rku.tmp'
Failed to download n03445777: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n03445777.tarelp7miio.tmp'
Failed to download n03376595: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n03376595.tar6d_fbiyf.tmp'
Failed to download n03584829: [Errno 2] No such file or directory: 'D:\\Vijay\\NYU\\Spring_25\\BDMLS\\Project\\dataset\\ImageNetEEG\\images\\n03584829.tarbtd0hb